# Tutorial about tracking LocData objects

Tracking refers to link localizations that are close in space over multiple frames and collect those localizations in individual tracks.
We here make use of the trackpy package through wrapper functions to deal with LocData objects.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Synthetic data

A random dataset is created.

In [ ]:
dat = lc.simulate_tracks(n_walks=5, n_steps=100, ranges=((0,1000),(0,1000)),
                      diffusion_constant=1, seed=1)

dat.print_meta()

In [ ]:
dat.data.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
dat.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Track locdata

The track function collects tracks in a new locdata object.

In [ ]:
tracks, track_numbers = lc.track(dat, search_range=500)

In [ ]:
tracks.print_summary()

In [ ]:
tracks.data

In [ ]:
tracks.references[0].data.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
dat.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
tracks.references[0].data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Red', label='track 0')
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
dat.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
tracks.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Red', label='tracks')
plt.show()

To show individual tracks make use of the reference attribute.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

jet= plt.get_cmap('jet')
colors = iter(jet(np.linspace(0, 1, len(tracks))))

for ref in tracks.references:
    c=next(colors)
    ref.data.plot.scatter(x='position_x', y='position_y', ax=ax, color=(c,), label=ref.meta.identifier)

plt.show()

Alternatively to a new locdata object, a pandas DataFrame can be generated using the link_locdata method.

In [ ]:
links = lc.link_locdata(dat, search_range=10)

In [ ]:
links.head()

Use the following to add particle column to original locdata dataset.

In [ ]:
dat.data.loc[links.index,'track']=links

In [ ]:
dat.data.head()

## Track using trackpy with locdata.data as input

For a detailed tracking analysis you might want to use [trackpy](https://soft-matter.github.io/trackpy) functions with the pandas DataFrame carrying localization data as input. The following examples will igve a short illustration of using trackpy functions.

In [ ]:
import trackpy as tp

In [ ]:
t = tp.link_df(dat.data, search_range=100, memory=1, pos_columns=['position_x', 'position_y'], t_column='frame')

t.head(10)

In [ ]:
plt.figure()
tp.plot_traj(t, pos_columns=['position_x', 'position_y'], t_column='frame');

## filter

In [ ]:
t.head()

In [ ]:
t1 = tp.filter_stubs(t, 10).reset_index(drop=True)
len(t1)

In [ ]:
plt.figure()
tp.plot_traj(t1, pos_columns=['position_x', 'position_y']);

## drift

In [ ]:
d = tp.compute_drift(t1, pos_columns=['position_x', 'position_y'])

In [ ]:
d.plot()
plt.show()

## Mean square displacement (msd)

In [ ]:
em = tp.emsd(t1,0.1, 100, pos_columns=['position_x', 'position_y']) # microns per pixel = 100/285., frames per second = 24

In [ ]:
fig, ax = plt.subplots()
ax.plot(em.index, em, 'o')
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
#ax.set(ylim=(1e-2, 10));